# Práctica 4 - Inteligencia Artificial

### Grado Ingeniería Informática Tecnologías Informáticas 

### Procesos de Decisión de Markov

José Luis Ruíz Reina

In [1]:
import random

En esta práctica vamos a implementar algoritmos relacionados con Procesos de Decisión de Markov. 

Supondremos que un Procesos de Decisión de Markov (MDP, por sus siglas en inglés) va a ser un objeto de la siguiente clase (o mejor dicho, de una subclase de la siguiente clase). 

In [2]:
class MDP(object):

    """La clase genérica MDP tiene como métodos la función de recompensa R,
    la función A que da la lista de acciones aplicables a un estado, y la
    función T que implementa el modelo de transición. Para cada estado y
    acción aplicable al estado, la función T devuelve una lista de pares
    (ei,pi) que describe los posibles estados ei que se pueden obtener al
    plical la acción al estado, junto con la probabilidad pi de que esto
    ocurra. El constructor de la clase recibe la lista de estados posibles y
    el factor de descuento.

    En esta clase genérica, las funciones R, A y T aparecen sin definir. Un
    MDP concreto va a ser un objeto de una subclase de esta clase MDP, en la
    que se definirán de manera concreta estas tres funciones"""  

    def __init__(self,estados,descuento):

        self.estados=estados
        self.descuento=descuento

    def R(self,estado):
        pass

    def A(self,estado):
        pass
        
    def T(self,estado,accion):
        pass

Consideramos el siguiente problema:

A lo largo de su vida, una empresa pasa por situaciones muy distintas, que por simplificar resumiremos en que al inicio de cada campaña puede estar rica o pobre, y ser conocida o desconocida.  Para ello puede decidir en cada momento o bien invertir en publicidad, o bien optar por no hacer publicidad. Estas dos acciones no tienen siempre un resultado fijo, aunque podemos describirlo de manera probabilística:

* Si la empresa es rica y conocida y no invierte en publicidad, seguirá rica, pero existe un 50% de probabilidad de que se vuelva desconocida. Si gasta en publicidad, con toda seguridad seguirá conocida pero pasará a ser pobre.  
* Si la empresa es rica y desconocida y no gasta en publicidad, seguirá desconocida, y existe un 50% de que se vuelva pobre. Si gasta en publicidad, se volverá pobre, pero existe un 50% de probabilidades de que se vuelva conocida.
* Si la empresa es pobre y conocida y no invierte en publicidad, pasará a ser pobre y desconocida con un 50% de probabilidad, y rica y conocida en caso contrario. Si gasta en publicidad, con toda seguridad seguirá en la misma situación. 
* Si la empresa es pobre y desconocida, y no invierte en publicidad, seguirá en la misma situación con toda seguridad. Si gasta en publicidad, seguirá pobre, pero con un 50% de posibilidades pasará aser conocida. 

Supondremos que la recompensa en una campaña en la que la empresa es rica es de 10, y de 0 en en las que sea pobre. El objetivo es conseguir la mayor recompesa acumulada a lo largo del tiempo, aunque penalizaremos las ganancias obtenidas en campañas muy lejanas en el tiempo, introduciendo un factor de descuento de 0.9. 

### Ejercicio 1

Se pide representar el problema como un proceso de decisión de Markov, definiendo una clase Rica_y_Conocida, subclase de la clase MDP genérica, cuyo constructor recibe como entrada únicamente el factor de descuento, y en la que se definen de manera concreta los métodos R, A y T, según lo descrito. 

Para ello:
* Los estados corresponderán a `"RC"`, `"RD"`, `"PC"` y `"PD"`. Y la función de recompensa asignaría 10 a `"RC"` y `"RD"`, y 0 a `"PC"` y `"PD"`.
* Las acciones serán: `"¬P"` y `"P"`.
* La transición será un diccionario donde las claves son tuplas (Estado, acción), `(s,a)` y los valores son listas de pares (Siguiente_Estado, Probabilidad),  `[(ns_1, P(ns_1 |s,a)), ..., (ns_k, P(ns_k |s,a))]`. Por ejemplo un par clave-valor del diccionario será `("RC","¬P"):[("RC",0.5),("RD":0.5)]`. Por eficiencia, puede ser recomendable guardarlo como atributo del problema.


#### Solución 

Lo primero que hemos de hacer es dar una representación al problema en el marco MDP. El siguiente diagrama de estados refleja tal modelización:

MDP_P4.svg

Ahora, es suficiente con trasladar el diagrama a la clase genérica MDP, especificando los elementos correspondientes: estados, función de recompensa, acciones aplicables a los estados, modelo de transición y factor de descuento.

In [14]:
# Solución:
class Rica_y_Conocida(MDP):
  def __init__(self, factor_descuento):
    super().__init__(["RC", "RD", "PC", "PD"], factor_descuento)
    self.Tdict = {
        ("RC", "P") : [("PC", 1.)],
        ("RC", "¬P") : [("RC", 0.5), ("RD", 0.5)],

        ("RD", "P") : [("PC", 0.5), ("PD", 0.5)],
        ("RD", "¬P") : [("RD",0.5),("PD",0.5)],
        
        ("PC", "P") : [("PC",1.)],
        ("PC", "¬P") : [("PD",0.5),("RC",0.5)],

        ("PD", "P") : [("PD",0.5),("PC",0.5)],
        ("PD", "¬P") : [("PD",1.)]
    }

  def R(self, estado):
    return 10. if estado[0] == 'R' else 0.

  def A(self, estado):
    return ["P", "¬P"]

  def T(self, estado, accion):
    return self.Tdict[(estado, accion)]

### Ejercicio 2

En general, dado un MDP, representaremos una política para el mismo como un diccionario cuyas claves son los estados, y los valores las acciones. Una política representa una manera concreta de decidir en cada estado la acción (de entre las aplicables a ese estado) que ha de aplicarse. 

Dado un MDP, un estado de partida, y una política concreta, podemos generar (muestrear) una secuencia de estados por los que iríamos pasando si vamos aplicando las acciones que nos indica la política: dado un estado de la secuencia, aplicamos a ese estado la acción que indique la política, y obtenemos un estado siguiente de manera aleatoria, pero siguiendo la distribución de probabilidad que indica el modelo de transición dado por el método `T`.  

Se pide definir una función `genera_secuencia_estados(mdp,pi,e,n)` que devuelva una secuencia de estados de longitud `n`, obtenida siguiendo el método anterior. Aquí mdp es objeto de la clase MDP, `pi` es una política, `e` un estado de partida y `n` la longitud de la secuencia. 

In [34]:
# Solución

def genera_secuencia_estados(mdp,pi,e,n):
  # El primer estado de la secuencia corresponde al estado inicial
  # indicado como entrada
  seq = [e]

  # Generar n-1 estados de forma aleatoria probabilística a partir del estado 
  # anterior de la secuencia generada
  for _ in range(n-1):
    # Calculamos una lista con los sucesores y sus probabilidades.
    # NOTA: Nótese que lo que devuelve la llamada al diccionario es la referencia,
    #       por lo que es necesario guardar una copia, ya que la vamos a modificar.
    cands = mdp.T(seq[-1], pi[seq[-1]]).copy()
    # Para elegir el siguiente estado con aleatoriedad probabilística, vamos a 
    # utilizar el método de la ruleta.
    #   1. Damos una ordenación a los sucesores (ya está hecho porque la lista 
    #      ya está ordenada)
    #   2. Calculamos la probabilidad acumulada para cada estado (en base a la 
    #      ordenación). 
    #      Esto es, si tuviesemos 
    #         [("S1", 0.1), ("S2", 0.3), ("S3", 0.2), ("S4", 0.4)]
    #      entonces la lista de probabilidades acumuladas correspondería a :
    #       [                         |    [
    #         ("S1", 0.1), -----------|----> ("S1", 0.1),
    #         ("S2", 0.3), -----------|----> ("S2", 0.1 + 0.3 = 0.4),
    #         ("S3", 0.2), -----------|----> ("S3", 0.1 + 0.3 + 0.2 = 0.6),
    #         ("S4", 0.4), -----------|----> ("S4", 0.1 + 0.3 + 0.2 + 0.4 = 1)
    #       ]                         |    ]
    #      Nótese que la suma de las probabilidades de los elementos anteriores 
    #      corresponde a la probabilidad acumulada del elemento inmediatamente 
    #      anterior.
    #
    #   3. Generamos un número aleatorio y tomamos el primer elemento cuya pro-
    #      babilidad acumulada es mayor que el número aleatorio

    cands_ac = [cands.pop(0)]
    while cands:
      cand, pcand = cands.pop(0)
      cands_ac += [(cand, cands_ac[-1][1] + pcand)]

    seq += [next(filter(lambda x: x[1] > random.random(), cands_ac))[0]]
  return seq

Vemos a continuación algunos ejemplo de uso:

In [35]:
# Definimos una instancia de la subclase 
mdp_ryc=Rica_y_Conocida(0.9)

In [36]:
pi_ryc_ahorra={"RC":"¬P","RD":"¬P", "PC":"¬P","PD":"¬P"}
genera_secuencia_estados(mdp_ryc,pi_ryc_ahorra,"PC",10)

# Posible resultado:
# ['PC', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD']

['PC', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD']

In [37]:
pi_ryc_2={"RC":"¬P","RD":"P", "PC":"¬P","PD":"P"}
genera_secuencia_estados(mdp_ryc,pi_ryc_2,"RD",8)

# Posible resultado:
# ['RD', 'PC', 'RC', 'RC', 'RC', 'RC', 'RD', 'PC']

['RD', 'PD', 'PD', 'PD', 'PC', 'PD', 'PC', 'RC']

### Ejercicio 3

Dado un MDP y una secuencia de estados, valoramos dicha secuencia como la suma de las recompensas de los estados de la secuencias (cada una con el correspondiente descuento). Se pide definir una función valora_secuencia(seq,mdp) que calcula esta valoración.

In [38]:
# Solución

def valora_secuencia(seq, mdp):
  # La expresión de valoración de una secuencia corresponde a: 
  #   V([q_0,q_1,...,q_k])= R(q_1) + 	ɤ·R(q_1) + ɤ^2·R(q_2) + ··· + ɤ^k·R(q_k)=
  #                       = ∑_{i in {0, 1, ..., k}} ɤ^i·R(q_i)
  return sum(mdp.descuento**i * mdp.R(s) for i, s in enumerate(seq))

In [39]:

valora_secuencia(['PC', 'RC', 'RC', 'RC', 'RC', 'RC', 
                       'RD', 'RD', 'RD', 'PD', 'PD', 'PD', 
                       'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 
                       'PD', 'PD'],mdp_ryc)

# Resultado:
# 51.2579511

51.2579511

In [40]:
valora_secuencia(['RD', 'PC', 'PD', 'PC', 'RC', 'RC', 
                        'RD', 'PD', 'PD', 'PC', 'RC', 'RC', 
                        'RC', 'RC', 'RC', 'RC'],mdp_ryc)

# Resultado:
# 44.11795212148159

44.11795212148159

### Ejercicio 4

Dada una política $\pi$, la valoración de un estado $s$ respecto de esa política, $V^{\pi}(s)$, se define como la media esperada de las valoraciones de las secuencias que se pueden generar teniendo dicho estado como estado de partida. Usando las funciones de los dos ejercicios anteriores, definir una función `estima_valor(s,pi,mdp,m,n)` que realiza una estimación aproximada del valor $V^{\pi}(s)$, para ello genera `n` secuencias de tamaño `m`, y calcula la media de sus valoraciones.  

In [43]:
# Solución:
def estima_valor(s, pi, mdp, m, n):
  # Se generan n secuencias de longitud m y se calcula su valor (con las funcio- 
  # nes de los dos ejercicios previos y se calcula la media)
  return sum(valora_secuencia(genera_secuencia_estados(mdp,pi,s,m), mdp) for i in range(n))/n

In [44]:
estima_valor("PC",pi_ryc_ahorra,mdp_ryc,50,500)

# Respuesta posible:
# 14.531471247172597

14.273670607263101

In [45]:
estima_valor("PC",pi_ryc_2,mdp_ryc,50,500)

# Respuesta posible:
# 35.92126959549151

34.82025053443127

In [46]:
estima_valor("RC",pi_ryc_ahorra,mdp_ryc,60,700)

# Respuesta posible:
# 32.807558694112984

33.01167528127202

In [47]:
estima_valor("RC",pi_ryc_2,mdp_ryc,60,700)

# Respuesta posible:
# 50.09728516585913

50.49961789456561

### Ejercicio 5

Usando la función anterior, estimar la valoración de cada estado del problema "Rica y conocida", con las dos siguientes políticas:

* Aquella que sea cual sea el estado, siempre decide invertir en publicidad. 
* Aquella que sea cual sea el estado, siempre decide ahorrar. 

¿Cuál crees que es mejor? ¿Habrá alguna mejor que estas dos? ¿Cuál crees que sería la mejor política de todas? 

In [49]:
# Solución
# Política que a todo estado le asigna la acción de invertir en publicidad
pi_ryc_gastar = {s : "P" for s in mdp_ryc.estados}

# Política que a todo estado le asigna la acción de no invertir en publicidad
pi_ryc_ahorra = {s : "¬P" for s in mdp_ryc.estados}

In [50]:
estima_valor("RC",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta aprox: 10.0

10.0

In [51]:
estima_valor("RC",pi_ryc_ahorra,mdp_ryc,60,1000)

# Respuesta aprox: 33.354461818277635

33.20167231436792

In [52]:
estima_valor("RD",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta aprox : 10.0

10.0

In [53]:
estima_valor("RD",pi_ryc_ahorra,mdp_ryc,60,1000)

# Respuesta aprox: 18.17532275274187

18.066925485999985

In [54]:
estima_valor("PC",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta aprox: 0.0

0.0

In [55]:
estima_valor("PC",pi_ryc_ahorra,mdp_ryc,60,1000)

# Respuesta aprox: 15.19963675853885

15.0129919798929

In [56]:
estima_valor("PD",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta aprox: 0.0

0.0

In [57]:
estima_valor("PD",pi_ryc_ahorra,mdp_ryc,60,1000)

# Respuesta aprox: 0.0

0.0

Se puede comprobar que, salvo en el último caso, la valoración es igual, las valoraciones que se consiguen con la segunda política son mayores que con la primera política. 

Ninguna de las dos políticas es la óptima, como se verá más adelante.

### Ejercicio 6

La función de valoración no se suele calcular mediante la técnica de muestreo vista en el ejercicio 4, sino como resultado de resolver un sistema de ecuaciones. Dicho sistema de ecuaciones se puede resolver de manera proximada de manera iterativa, tal como se explica en el tema.

Definir una función `valoración_respecto_política(pi,mdp, n)` que aplica dicho método iterativo ($n$ iteraciones) para calcular la valoración $V^{\pi}$. Dicha valoración debe devolverse como un diccionario que a cada estado $s$ le asocia el valor $V^{\pi}(s)$ calculado.  

Aplicar la función para calcular la valoración asociada a las dos políticas que se dan en el ejercicio anterior, y comparara los valores obtenidos con los obtenidos mediante muestreo. 

In [62]:
# Solución:
def valoración_respecto_política(pi,mdp, n):
  # El esquema del método iterativo de aproximación de V^π corresponde a:
  #   1. Se genera unos valores aleatorios para cada uno de los estados (o 0s)
  Vpi = dict([(s, random.random()) for s in mdp.estados]) 
        # o dict([(s, 0.) for s in mdp.estados])
  #   2. Hasta la condición de parada (en este caso un número prefijado de n 
  #      iteraciones)
  for _ in range(n):
    # 2.1 Para cada estado V^π(s) <- R(s) + ɤ· ∑_{s'}(P(s'|s,a)·V^π(s'))
    #     Esta acción ha de hacerse en paralelo bien haciéndolo utilizando las
    #     características funcionales de python
    Vpi = dict([(s, mdp.R(s) + mdp.descuento*sum(map(lambda x: x[1]*Vpi[x[0]], mdp.T(s, pi[s]))))  for s in mdp.estados])

    # O bien realizando una copia e ir actualizando cada estado por separado:
    # Vpi_ = Vpi.copy()
    # for s in mdp.estados:
    #   Vpi_[s] = mdp.R(s) + mdp.descuento*sum(map(lambda x: x[1]*Vpi[x[0]], mdp.T(s, pi[s])))
    # Vpi = Vpi_
  return Vpi

Calculamos con esta función la valoración de las dos políticas anteriores.

In [69]:
valoración_respecto_política(pi_ryc_gastar,mdp_ryc, 1000)

# Resultado:
# {'RC': 10.0, 'RD': 10.0, 'PC': 0.0, 'PD': 0.0}

{'RC': 10.0,
 'RD': 10.0,
 'PC': 1.5707993223993104e-46,
 'PD': 1.5707993223993104e-46}

In [70]:
valoración_respecto_política(pi_ryc_ahorra,mdp_ryc, 1000)

# Resultado:
# {'RC': 33.05785123966942,
#  'RD': 18.18181818181818,
#  'PC': 14.876033057851238,
#  'PD': 0.0}

{'RC': 33.05785123966943,
 'RD': 18.181818181818187,
 'PC': 14.876033057851245,
 'PD': 1.130387128985454e-46}

### Ejercicio 7

En el tema 3 se ha visto que la valoración de un estado se define como la mejor valoración que pueda tener el estado, respecto a todas las políticas posibles. Y la política óptima es aquella que en cada estado realiza la acción con la mejor valoración esperada (entendiendo por valoración esperada la suma de las valoraciones de los estados que podrían resultar al aplicar dicha acción, ponderadas por la probabilidad de que ocurra eso). De esta manera, la valoración de un estado es la valoración que la política óptima asigna al estado.

Para calcular tanto la valoración de los estados, como la política óptima, se han visto dos algoritmos: iteración de valores e iteración de políticas. En este ejercicio se pide implementar el algoritmo de iteración de políticas. En concreto, se pide definir una función `iteración_de_políticas(mdp,k)` que implementa el algoritmo de iteración de políticas, y devuelve dos diccionarios, uno con la valoración de los estados y otro con la política óptima. 

Comparar los resultados obtenidos con las políticas del ejercicio 5 y las valoraciones obtenidas.  

In [71]:
# Solución:
def iteración_de_políticas(mdp,k):
  # Siguiendo el esquema algorítmico de Iteración de políticas:
  #   1. Se genera una política inicial, eligiendo aleatoriamente, para cada 
  #      estado, una acción de entre las que son aplicables a dicho estado.
  pi_star = dict([(s, random.choice(mdp.A(s))) for s in mdp.estados])
  
  #   2. Mientras haya una actualización en la política (considerando inicial-
  #      mente que ha sido modificada, para que entre en el proceso iterativo)
  actualizada = True
  while actualizada:
  #   3. Calcular la valoración de cada cada con respecto a la política actual
    V = valoración_respecto_política(pi_star, mdp, k)
  
  #   4. Dados los valores de los estados V, comprobar para cada estado si la
  #      la recompensa media esperada con otra acción distinta a la marcada por
  #      política es mejor que la esperada por la acción de la política. 
  #     
  #      En caso positivo, actualizar la política sustituyendo la acción asociada 
  #      al estado por la de mejor recompensa media esperada, y marcarla como 
  #      actualizada.
    actualizada = False
    for s in mdp.estados:
      
      # Calculamos la recompensa esperada para cada acción distinta de la marcada
      # por la política (en realidad faltaría sumar la recompensa del estado y
      # aplicar el factor de descuento, pero es irrelevante, ya que a todos les
      # correspondería esa misma transformación). Tomamos la de mejor recompensa
      # media esperada. 
      cand = max([(a, sum(map(lambda x: x[1]*V[x[0]], mdp.T(s,a)))) for a in mdp.A(s) if a != pi_star[s]], key=lambda x:x[1])
      # Si la recompensa media esperada de la mejor candidata es mejor que la 
      # correspondiente a la acción marcada por la política:
      if cand[1] > sum(map(lambda x: x[1]*V[x[0]], mdp.T(s,pi_star[s]))):
        # Entonces actualizar la acción asociada al estado con la de la mejor 
        # candidata.Marcar la política como actualizada
        pi_star[s] = cand[0]
        actualizada = True
      # En otro caso seguir con el siguiente estado
  
  # Si la política no se actualiza entonces ya hemos encontrado la política 
  # óptima, y de paso, el valor de los estados.
  return pi_star, V



Calculamos la mejor política y su valoración con el MDP de Rica_y_Conocida. Como se ve, resulta que lo mejor es sólo gastar en publicidad cuando se es pobre y desconocido. Se observa que la valoración respecto de esa política es mejor, que las valoraciones con las política que se vieron en los ejercicios 5 y 6.   

In [72]:
iteración_de_políticas(mdp_ryc,100)

# Respuesta
# ({'RC': 'No publicidad',
#   'RD': 'No publicidad',
#   'PC': 'No publicidad',
#   'PD': 'Gastar en publicidad'},
#  {'RC': 54.20053629623792,
#   'RD': 44.02311379672535,
#   'PC': 38.602953921506,
#   'PD': 31.584041852876634})

({'RC': '¬P', 'RD': '¬P', 'PC': '¬P', 'PD': 'P'},
 {'RC': 54.200550905731355,
  'RD': 44.02312840621879,
  'PC': 38.60296853099944,
  'PD': 31.584056462370075})